In [1]:
import os
os.chdir('/home/ubuntu/Desktop/DL_WorkSpace/WorkSpace/Input_Data_Container/Bangkok_Province/Commercial_WS')

In [2]:
home_direc = os.getcwd()
print(home_direc)

/home/ubuntu/Desktop/DL_WorkSpace/WorkSpace/Input_Data_Container/Bangkok_Province/Commercial_WS


In [3]:
import os, shutil, sys
import glob
from osgeo import gdal
# from gdalconst import GA_ReadOnly
from tqdm.auto import tqdm


def remove_path(file_list):
    fl_list = []
    for fl in file_list:
        head, filename = os.path.split(fl)
        fl_list.append(filename)
    return fl_list

def get_selected_files(input_dir_path, ext='.tif'):
    'Returns files with an extension'
    input_images = os.path.join(input_dir_path, '*'+ext)
    file_list_with_path = [f for f in glob.glob(input_images)]
    file_list = remove_path(file_list_with_path)
    return file_list

# 获取遥感图片的坐标，投影和分辨率
# Get the coordinates, projection and resolution of remote sensing pictures
def get_raster_extent(ref_img):
    #print('ref_img', ref_img)
    
    maskDs = gdal.Open(ref_img , gdal.GA_ReadOnly)# your mask raster
    projection=maskDs.GetProjectionRef()
    geoTransform = maskDs.GetGeoTransform()
    minx = geoTransform[0]
    maxy = geoTransform[3]
    maxx = minx + geoTransform[1] * maskDs.RasterXSize
    miny = maxy + geoTransform[5] * maskDs.RasterYSize
    
    x_resolution =  geoTransform[1] 
    y_resolution =  geoTransform[5]  
    
    return minx, miny, maxx, maxy, projection, x_resolution, y_resolution

def crop_raster(minx, miny, maxx, maxy, projection, target_img, output_img):
    #output_img =  ref_img.replace('tif', 'png')
    data = gdal.Open(target_img , gdal.GA_ReadOnly) #Your data the one you want to clip
    gdal.Translate(output_img,  data,  format='GTiff', projWin=[minx, maxy, maxx, miny],   outputSRS=projection  , creationOptions = ['TFW=YES', 'COMPRESS=Deflate']) 
    return

def convert_resolution(in_img, x_res_target, y_res_target, final_op):
    #in_img = in_dir + img
    file_name = os.path.basename(in_img)
    cmd = 'gdalwarp -tr '+ str(x_res_target )  +' '+  str(y_res_target) +' '+ in_img +' ' + final_op
#     print(cmd)
    os.system(cmd)
    return final_op

#remove those images which do not have respective masks (and vice versa)
def match_and_clean_img_mask(mask_dir, img_dir):
    arr_m = get_selected_files(mask_dir, ext='.tif')
    arr_i = get_selected_files(img_dir, ext='.tif')

    arr_excess_img = set(arr_i)- set(arr_m)   
    #print('arr_excess_img', arr_excess_img)
    for ii in arr_excess_img:
        os.remove(os.path.join(img_dir, ii))
    return 

def generate_image_mask_pair(raw_img_dir, mask_dir, img_dir):
    
    temp_dir = 'junk/'
    os.makedirs(temp_dir, exist_ok = True)

    for img in tqdm(arr_img):
        final_op_mask = os.path.join(mask_dir, img)   # mask文件夹 = Mask folder
        final_op_img = os.path.join(img_dir, img)    # 输出图片目录 = Output image directory
        img_path = os.path.join(raw_img_dir, img)   # 输入图像目录 = Enter image directory

        # get image extent
        minx,miny, maxx, maxy, projection, x_resolution, y_resolution = get_raster_extent(img_path)
        #print('x_resolution, y_resolution ' , x_resolution, y_resolution )

        #crop the geojson for that extent
        temp_geojson = os.path.join(temp_dir, 'temp.geojson')
        extent_str = ' ' + str(minx) +'  '+  str(miny) +'  '+ str(maxx) +'  '+ str(maxy) +'  '
        crop_cmd = 'ogr2ogr  -spat '+ extent_str +' -f GeoJSON ' + temp_geojson +' ' + ip_geojson
        os.system(crop_cmd)
        #print(crop_cmd)

        #gdal_rasterize the geojson to geotiff    
        temp_raster = os.path.join(temp_dir, 'temp.tif')
        tr_str =  ' '+ str(x_resolution) +' '+ str( y_resolution)+' '
        gdal_cmd = 'gdal_rasterize  -co \'COMPRESS=DEFLATE\' -burn 1 -tr '+ tr_str + temp_geojson + ' '+ temp_raster
        os.system(gdal_cmd)
        #print(gdal_cmd)

        # crop by  reference image extent 
        temp_raster_clipped = temp_dir + 'temp_clipped.tif'
        gdal_warp = 'gdalwarp  -co \'COMPRESS=DEFLATE\'  -te '+ extent_str + ' '+ temp_raster +' '+ final_op_mask # temp_raster_clipped
        os.system(gdal_warp) #
        #print(gdal_warp)

        shutil.copyfile(img_path, final_op_img)
    
    # 删除临时文件夹 = Delete the temporary folder
    shutil.rmtree(temp_dir) 
    return

In [4]:
import os
from tqdm import tqdm

test_img_dir = os.path.join(home_direc, 'yolo8_segmentation_ws', 'test/images')
img_list = [f for f in os.listdir(test_img_dir) if f.endswith('.jpg')]

req_img_dir = os.path.join(home_direc, 'scene_classification_ws', 'tiff')  # Path to the original image directory
new_folder =  os.path.join(home_direc, 'yolo8_segmentation_ws', 'test_dataset_pairs')  # Path to the new folder where file paths will be saved

# Create the new folder if it doesn't exist
if not os.path.exists(new_folder):
    os.makedirs(new_folder)

for img in tqdm(img_list):
    img_name = img.split('.')[0][:-4]
    img_path = os.path.join(req_img_dir, img_name + '.tif')
    
    # Get the filename without the directory
    img_filename = os.path.basename(img_path)
    
    # Create the new file path in the new folder
    new_path = os.path.join(new_folder, img_filename)
    
    # Copy the file to the new folder
    shutil.copyfile(img_path, new_path)

100%|████████████████████████████████████████| 779/779 [00:06<00:00, 122.36it/s]


In [5]:
#GeneratesTIFF Image and Binary Mask of Geojson Pairs for each image

raw_img_dir = os.path.join(home_direc, 'yolo8_segmentation_ws', 'test_dataset_pairs')    
ip_geojson  = os.path.join(home_direc, 'scene_classification_ws', 'Commercial_BF.geojson')

# Get the input picture list
arr_img = get_selected_files(raw_img_dir)
mask_dir = os.path.join(home_direc, 'yolo8_segmentation_ws', 'test_dataset_pairs', 'test_gt_mask')
img_dir = os.path.join(home_direc, 'yolo8_segmentation_ws', 'test_dataset_pairs', 'test_gt_img')
os.makedirs(mask_dir, exist_ok = True) 
os.makedirs(img_dir, exist_ok = True)

generate_image_mask_pair(raw_img_dir, mask_dir, img_dir)
#  remove those images which do not have respective masks (and vice versa)
match_and_clean_img_mask(mask_dir, img_dir)

  0%|                                         | 1/779 [00:29<6:28:22, 29.95s/it]

0...10...20...30...40...50...60...70...80...90...100 - done.
Creating output file that is 640P x 640L.
Processing junk/temp.tif [1/1] : 0...10...20...30...40...50...60...70...80...90...100 - done.


  0%|                                         | 2/779 [00:58<6:17:20, 29.14s/it]

0...10...20...30...40...50...60...70...80...90...100 - done.
Creating output file that is 640P x 640L.
Processing junk/temp.tif [1/1] : 0...10...20...30...40...50...60...70...80...90...100 - done.


  0%|▏                                        | 3/779 [01:27<6:14:19, 28.94s/it]

...

0...10...20...30...40...50...60...70...80...90...100 - done.
Creating output file that is 640P x 640L.
Processing junk/temp.tif [1/1] : 0...10...20...30...40...50...60...70...80...90...100 - done.


100%|██████████████████████████████████████▉| 778/779 [6:13:02<00:28, 28.94s/it]

0...10...20...30...40...50...60...70...80...90...100 - done.
Creating output file that is 640P x 640L.
Processing junk/temp.tif [1/1] : 0...10...20...30...40...50...60...70...80...90...100 - done.


100%|███████████████████████████████████████| 779/779 [6:13:31<00:00, 28.77s/it]

0...10...20...30...40...50...60...70...80...90...100 - done.
Creating output file that is 640P x 640L.
Processing junk/temp.tif [1/1] : 0...10...20...30...40...50...60...70...80...90...100 - done.
